In [80]:
import clang.cindex
from clang.cindex import CursorKind
from anytree import RenderTree
from tree_node import Node
import utils
from node_handler import *
import re
import subprocess
import pexpect
from tqdm import tqdm
import re
import pandas as pd
import os
import subprocess
import pexpect


In [84]:
program = """#define ONLINE_JUDGE 
// program to add two numbers using a function

#include <iostream>

using namespace std;

// declaring a function
int add(int a, int b) {
    return (a + b);
}"""


p = subprocess.Popen(['g++', '-x', 'c++', '-Wall', '-o', 'myprog' '/dev/stdin'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
p.communicate(bytes(program))

TypeError: string argument without an encoding

In [101]:
analyzer = pexpect.spawn('g++ -x c++ -Wall -E /dev/stdin', encoding='utf-8')
analyzer.sendline(program)
p.expect(pexpect.EOF)
analyzer.read()

In [61]:
subprocess.check_output(['echo', program, '|', 'g++', '-x', 'c++', '-Wall', '-E', '/dev/stdin'])

b'#define ONLINE_JUDGE \n// program to add two numbers using a function\n\n#include <iostream>\n\nusing namespace std;\n\n// declaring a function\nint add(int a, int b) {\n    return (a + b);\n} | g++ -x c++ -Wall -E /dev/stdin\n'

In [9]:
def preprocess_solution(row):
    solution = row['solution']
    includes_usings = []
    include_regex = re.compile(r'^#*.include')
    using_regex = re.compile(r'^using')
    lines = re.split(';|\n', solution)
    includes_usings += [element + ';' for element in list(filter(using_regex.search, lines))]
    includes_usings += list(filter(include_regex.search, lines))


    if not '#define ONLINE_JUDGE' in solution:
        solution = '#define ONLINE_JUDGE \n' + solution

    return solution, includes_usings

In [10]:
input_file = '../codeforces-scraper/data/contests_solutions/solutions_600.csv'
output_file = '../data/cpp_preprocessed/solutions_600.csv'

header = True
df = pd.read_csv(input_file, chunksize=5e5)

for chunk in df:
    chunk = chunk[chunk['programmingLangauge'].str.contains('C\+\+')]
    chunk = chunk[~chunk['solution'].isnull()]
    tqdm.pandas()
    chunk[['solution','imports']] = chunk.progress_apply(preprocess_solution, axis=1, result_type='expand')
    chunk.to_csv(output_file, header=header, index=False, mode='a')
    header = False

100%|██████████| 375984/375984 [00:56<00:00, 6691.47it/s] 
/home/sander/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
100%|██████████| 303187/303187 [00:44<00:00, 6800.08it/s] 
0it [00:00, ?it/s]


ValueError: Columns must be same length as key

In [11]:
data = pd.read_csv('../data/cpp_preprocessed/solutions_600.csv')

In [43]:
for program in data[['solution', 'solutionId']].iterrows():
    print(program[1])
    break

solution      #define ONLINE_JUDGE \n///Fix the cause, not t...
solutionId                                            107004724
Name: 0, dtype: object


In [14]:
data.drop(columns='Unnamed: 0', inplace=True)

In [16]:
data.to_csv('../data/cpp_preprocessed/solutions_600.csv', index=False)

In [44]:
data[:100].to_csv('../data/cpp_preprocessed/test.csv', index=False)

In [207]:
solutions[solutions['solutionId'] == 104554251]

,solutionId,contestId,problem,programmingLanguage
106,104554251,1070,B,GNU C++11


In [47]:
import clang.cindex

In [50]:
clang_lib_file='/usr/lib/x86_64-linux-gnu/libclang-6.0.so.1'
clang.cindex.Config.set_library_file(clang_lib_file)

In [51]:
index = clang.cindex.Index.create()

In [66]:
index.parse('tmp.cpp', unsaved_files=[('tmp.cpp', program)], args=['-x', 'c++', '-std=c++17'])

In [55]:
program = data.iloc[0]['solution']

In [56]:
program.replace('\n', '')

'#define ONLINE_JUDGE ///Fix the cause, not the symptom.#include<bits/stdc++.h>using namespace std;#define ll           long long#define ull          unsigned long long#define db           double#define sf           scanf#define pf           printf#define pb           push_back#define pi           acos(-1)#define MOD          1000000007#define pii          pair<ll,ll>#define time_save    ios_base::sync_with_stdio(false); cin.tie(NULL);#define D(x)         cerr << __LINE__ << ": " << #x << " = " << (x) << \'\\n\'#define DD(x,y)      cerr << __LINE__ << ": " << #x << " = " << x << "   " << #y << " = " << y << \'\\n\'#define DDD(x,y,z)   cerr << __LINE__ << ": " << #x << " = " << x << "   " << #y << " = " << y << "   " << #z << " = " << z << \'\\n\'#define DBG          cerr << __LINE__ << ": Hi" << \'\\n\'/*----------------------Graph Moves----------------*///const ll dx[]={+1,-1,+0,+0};//const ll dy[]={+0,+0,+1,-1};//const ll dx[]={+0,+0,+1,-1,-1,+1,-1,+1};   // Kings Move//const ll dy[]

In [6]:
class AST_parser:
    def __init__(self, clang_lib_file='/usr/lib/x86_64-linux-gnu/libclang-6.0.so.1'):
        try:
            clang.cindex.Config.set_library_file(clang_lib_file)
        except Exception as e:
            print(f'Skipped setting library file: {e}')

        self.index = clang.cindex.Index.create()


    def parse_ast(self, input_file_path):
        tu = self.index.parse(input_file_path)
        cursor_items = self.get_cursor_items(tu.cursor)
        root_node = Node('root', is_reserved=True)

        for cursor_item in cursor_items:
            self.parse_item(cursor_item, root_node)

        return root_node


    def get_cursor_items(self, cursor):
        cursor_items = []
        user_code_started = False
        root_node = Node('root', True)

        for child in cursor.get_children():
            if user_code_started:
                cursor_items.append(child)
            if child.kind == CursorKind.USING_DIRECTIVE:
                user_code_started = True

        return cursor_items


    def parse_item(self, ast_item, parent_node):
        # skip meaningless AST primitives
        if ast_item.kind == CursorKind.DECL_STMT or ast_item.kind == CursorKind.UNEXPOSED_EXPR \
            or ast_item.kind == CursorKind.TEMPLATE_REF or ast_item.kind == CursorKind.NAMESPACE_REF:
            pass


        # Parse typdef
        elif utils.is_typedef(ast_item):
            handle_typedef(ast_item, parent_node)


        # parse declaration
        elif ast_item.kind.is_declaration():
            parent_node = handle_declaration(ast_item, parent_node, self.parse_item)

        # parse operator
        elif utils.is_operator(ast_item):
            parent_node = handle_operator(ast_item, parent_node)

        # parse literal
        elif utils.is_literal(ast_item):
            handle_literal(ast_item, parent_node)


        # parse call expression
        elif utils.is_call_expr(ast_item):
            parent_node = handle_call_expr(ast_item, parent_node)


        # parse reference
        elif utils.is_reference(ast_item):
            handle_reference(ast_item, parent_node)
        

        # parse type ref
        elif ast_item.kind == CursorKind.TYPE_REF and parent_node.label != 'root':
            handle_type_ref(ast_item, parent_node)


        # if not one of the above -> create simple parent node of the kind of the item
        else:
            parent_node = Node(ast_item.kind.name, is_reserved=True, parent=parent_node)


        for child in ast_item.get_children():
            if child.kind != CursorKind.PARM_DECL:
                self.parse_item(child, parent_node)

In [7]:
ast_parser = AST_parser()
ast = ast_parser.parse_ast('../data/subset/cpp_preprocessed/104558927.cpp')

Skipped setting library file: library file must be set before before using any other functionalities in libclang.


NameError: name 'CursorKind' is not defined

In [ ]:
for pre, fill, node in RenderTree(ast):
    treestr = u"%s%s" % (pre, node.label)
    print(treestr)

In [ ]:
from anytree.exporter import JsonExporter

exporter = JsonExporter(indent=2)
with open('tree.json', 'w') as file:
    file.write(exporter.export(root_node))


In [124]:
for _,_,files in os.walk('../data/subset/ast_trees_to_code/'):
    print(files)

['104554194.cpp', '104554251.cpp', '104558927.cpp', '104560220.cpp', '104604737.cpp', '104606100.cpp', '104663473.cpp', '104696038.cpp', '104759177.cpp', '104759215.cpp', '104760965.cpp', '104766018.cpp', '104930101.cpp', '104931090.cpp', '104939468.cpp', '105028826.cpp', '105052007.cpp', '105066757.cpp', '105085785.cpp', '105087511.cpp', '105087586.cpp', '105113472.cpp', '105121226.cpp', '105129923.cpp', '106067757.cpp', '106076703.cpp', '106076991.cpp', '106079532.cpp', '106079587.cpp', '106081350.cpp', '106081812.cpp', '106085118.cpp', '106098695.cpp', '106181322.cpp', '106187233.cpp', '106188303.cpp', '106207179.cpp', '106209681.cpp', '106210901.cpp', '106210928.cpp', '106219172.cpp', '106283598.cpp', '106293015.cpp', '106295849.cpp', '106389953.cpp', '106395892.cpp', '106456255.cpp', '106461660.cpp', '106504120.cpp', '106508975.cpp', '106509000.cpp', '106534201.cpp', '106612149.cpp', '106637474.cpp', '106643718.cpp', '106696279.cpp', '106697152.cpp', '106701857.cpp', '106705445.cp

In [141]:
solution = 104554194


solutions[solutions['solutionId'] == solution]
solution_data = solutions[solutions['solutionId'] == solution]

tests = problems[(problems['contestId'] == solution_data['contestId'].iloc[0]) & (problems['problem'] == solution_data['problem'].iloc[0])]['allTests'].iloc[0]
print(solution_data)
tests[:500]

     solutionId  contestId problem programmingLanguage
107   104554194       1070       B           GNU C++11


"[('1\\n-149.154.167.99', '1\\n0.0.0.0/0'), ('4\\n-149.154.167.99\\n+149.154.167.100/30\\n+149.154.167.128/25\\n-149.154.167.120/29', '2\\n149.154.167.96/30\\n149.154.167.112/28'), ('5\\n-127.0.0.4/31\\n+127.0.0.8\\n+127.0.0.0/30\\n-195.82.146.208/29\\n-127.0.0.6/31', '2\\n127.0.0.4/30\\n128.0.0.0/1'), ('2\\n+127.0.0.1/32\\n-127.0.0.1', '-1'), ('1\\n-0.0.0.0/32', '1\\n0.0.0.0/0'), ('1\\n-0.0.0.10', '1\\n0.0.0.0/0'), ('1\\n-0.0.0.1', '1\\n0.0.0.0/0'), ('1\\n-0.0.0.13/32', '1\\n0.0.0.0/0'), ('1\\n-0.0.0.7', '1\\n0.0.0.0/0'),"

In [221]:
print(len(not_compiled))
print(len(tests_failed))
print(len(success))


0
15
3337


In [225]:
len(set(success))

86